## 뉴스 Crawling ㄱㄱ

In [ ]:
# 일단 다 긁어온거라 추후에 필요없는건 삭제 필요요

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import math


# --------------------------------------
# 웹페이지 열고 이동하기 
# --------------------------------------
driver = Chrome()

# 브라우저 크기 최대화
driver.maximize_window()

In [36]:
# --------------------------------------
# 종목 코드 이용하여 주식 뉴스 페이지 들어가기
# --------------------------------------
stockCode = '005930' 	# 삼성전자 종목 코드
startPage = 1			# 뉴스 수집 시작 페이지
lastPage = 5 			# 뉴스 수집 마지막 페이지

In [ ]:
# news_data 초기화
news_data = []

# 시작 페이지부터 마지막 페이지까지 순서대로 뉴스 데이터 수집집
for pageNum in range(startPage, lastPage + 1):
	url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
	driver.get(url)

	# iframe이 로드될 때까지 대기
	try: 
		WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "news_frame")))

		# iframe으로 전환 (name 또는 id 사용 가능)
		driver.switch_to.frame("news_frame") # 여기선 id 사용

		# 모든 tr 선택
		rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

		for i, row in enumerate(rows):
			# tr의 class 속성 가져오기
			tr_class = row.get_attribute("class")
			if tr_class and "relation_lst_cluster" in tr_class:
				continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨

			tds = row.find_elements(By.TAG_NAME, "td")
			if len(tds) >= 3:
				try:
					a_tag = tds[0].find_element(By.TAG_NAME, "a")
					newsTitle = a_tag.text
					newsAddress = a_tag.get_attribute("href")
				except:
					newsTitle = ""
					newsAddress = ""
				newsFrom = tds[1].text
				newsDate = tds[2].text
				#print(newsTitle, newsFrom, newsDate)
				news_data.append({'title': newsTitle, 'from': newsFrom, 'date': newsDate, 'link' : newsAddress})


		print(f"페이지 {pageNum} 완료 - 현재까지 {len(news_data)}개 뉴스")

		
	except Exception as e:
		print(f"페이지 {pageNum}에서 오류 발생: {e}")

	finally:
        # 다음음메인 프레임으로 복귀 
		driver.switch_to.default_content()

df = pd.DataFrame(news_data)
df

페이지 1 완료 - 현재까지 12개 뉴스
페이지 2 완료 - 현재까지 28개 뉴스
페이지 3 완료 - 현재까지 42개 뉴스
페이지 4 완료 - 현재까지 59개 뉴스
페이지 5 완료 - 현재까지 71개 뉴스


,title,from,date,link
0,옛 남산힐튼-서울역 인근 개발 사업 탄력···‘이오타 서울’ 대형기.....,매일경제,2025.05.30 15:05,https://n.news.naver.com/mnews/article/009/000...
1,자율주행 전기차 무선 자동충전 후 이동주차 '실증',연합뉴스,2025.05.30 15:00,https://n.news.naver.com/mnews/article/001/001...
2,"김문수, 'MS노믹스' 발표…""일자리 중심 성장·복지"" [대선 2025...",SBS Biz,2025.05.30 14:59,https://n.news.naver.com/mnews/article/374/000...
3,[주식 초고수는 지금]'100조 클럽' 현대HD 순매수 1위,서울경제,2025.05.30 14:49,https://n.news.naver.com/mnews/article/011/000...
4,"TSMC 손 잡은 SK하이닉스, HBM4 로직 다이 비용 압박↑",지디넷코리아,2025.05.30 14:33,https://n.news.naver.com/mnews/article/092/000...
...,...,...,...,...
66,"외인·기관 동반매도에…코스피, 2710선 출발 [개장시황]",뉴스1,2025.05.30 09:28,https://n.news.naver.com/mnews/article/421/000...
67,엔비디아 상승세 못 따라간 SK하이닉스…'21만닉스' 반납[핫종목],뉴스1,2025.05.30 09:27,https://n.news.naver.com/mnews/article/421/000...
68,약보합 출발 코스피…2700선 지키나,아시아경제,2025.05.30 09:22,https://n.news.naver.com/mnews/article/277/000...
69,코스피·코스닥 동반 하락 출발…'트럼프 관세' 불확실성에 발목,이데일리,2025.05.30 09:18,https://n.news.naver.com/mnews/article/018/000...


In [52]:
# DataFrame을 csv 파일로 출력력

uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_news.csv'
#file_path_excel = f'data/{prefix}_news.xlsx'

df.to_csv(file_path_csv)
print(f"{startPage}페이지부터 {lastPage}페이지까지 총 {lastPage - startPage + 1}페이지에서 {(len(df))}개의의 뉴스 기사를 CSV 파일({file_path_csv})로 저장하였습니다.")

1페이지부터 5페이지까지 총 5페이지에서 71개의의 뉴스 기사를 CSV 파일(data/20250530_151749_news.csv)로 저장하였습니다.


In [ ]:
### 여기서부턴 초기버전 코드. 필요 X 

In [ ]:
url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
driver.get(url)

In [ ]:
# iframe이 로드될 때까지 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "news_frame"))
)

<selenium.webdriver.remote.webelement.WebElement (session="44f9319500ffa0888467e7681d5d8ff1", element="FE895357DCD2290E9C7771562A5C0952_element_5402")>

In [25]:
# iframe으로 전환 (name 또는 id 사용 가능)
driver.switch_to.frame("news_frame")  


In [28]:
# 모든 tr 선택
rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

for i, row in enumerate(rows):
    # tr의 class 속성 가져오기
    tr_class = row.get_attribute("class")
    if tr_class and "relation_lst_cluster" in tr_class:
        continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨

    tds = row.find_elements(By.TAG_NAME, "td")
    if len(tds) >= 3:
        try:
            newsTitle = tds[0].find_element(By.TAG_NAME, "a").text
        except:
            newsTitle = ""
        newsFrom = tds[1].text
        newsDate = tds[2].text
        print(newsTitle, newsFrom, newsDate)
        news_data.append({'title': newsTitle, 'from': newsFrom, 'date': newsDate})


news_data

삼성, 역성장 중남미 스마트폰 시장서도 1위 수성 조선비즈 2025.05.30 14:29
관세 충격 피한 북미 TV 시장…삼성·LG 일단 안도 이데일리 2025.05.30 14:17
[백종민의 딥테크]삼성이 포기한 서버로 富일군 대만 아시아경제 2025.05.30 14:14
삼성 스마트워치 1분기 출하량 전년比 74%↑… 저가·고가 제품 ‘투 ... 조선비즈 2025.05.30 14:07
코스피, 외인 매도에 1%대 하락…코스닥도 재차 약세 이데일리 2025.05.30 13:56
'대어' 압구정2 놓고 삼성 vs 현대 재격돌…수주전 본격화(종합) 연합뉴스 2025.05.30 13:17
‘국제학교 품은 학세권’ ‘평택 고덕 A48블록 금성백조 예미지’에..... 서울경제 2025.05.30 12:59
창업·스타트업 지원 공약…지식산업센터 호재될까 매일경제 2025.05.30 12:59
"월급이 의느님 2배"...'삼성 고시'에 베트남 전국 수재들 다 모였... 파이낸셜뉴스 2025.05.30 12:08
코스피 5000, 아주 불가능한 건 아니다? 대통령 누가돼도 우상향이라... YTN 2025.05.30 11:52


[{'title': '삼성, 역성장 중남미 스마트폰 시장서도 1위 수성',
  'from': '조선비즈',
  'date': '2025.05.30 14:29'},
 {'title': '관세 충격 피한 북미 TV 시장…삼성·LG 일단 안도',
  'from': '이데일리',
  'date': '2025.05.30 14:17'},
 {'title': '[백종민의 딥테크]삼성이 포기한 서버로 富일군 대만',
  'from': '아시아경제',
  'date': '2025.05.30 14:14'},
 {'title': '삼성 스마트워치 1분기 출하량 전년比 74%↑… 저가·고가 제품 ‘투 ...',
  'from': '조선비즈',
  'date': '2025.05.30 14:07'},
 {'title': '코스피, 외인 매도에 1%대 하락…코스닥도 재차 약세',
  'from': '이데일리',
  'date': '2025.05.30 13:56'},
 {'title': "'대어' 압구정2 놓고 삼성 vs 현대 재격돌…수주전 본격화(종합)",
  'from': '연합뉴스',
  'date': '2025.05.30 13:17'},
 {'title': '‘국제학교 품은 학세권’ ‘평택 고덕 A48블록 금성백조 예미지’에.....',
  'from': '서울경제',
  'date': '2025.05.30 12:59'},
 {'title': '창업·스타트업 지원 공약…지식산업센터 호재될까',
  'from': '매일경제',
  'date': '2025.05.30 12:59'},
 {'title': '"월급이 의느님 2배"...\'삼성 고시\'에 베트남 전국 수재들 다 모였...',
  'from': '파이낸셜뉴스',
  'date': '2025.05.30 12:08'},
 {'title': '코스피 5000, 아주 불가능한 건 아니다? 대통령 누가돼도 우상향이라...',
  'from': 'YTN',
  'date': '2025.05.30 11:52'},
 {'title': '삼

In [33]:
df = pd.DataFrame(news_data)
df

,title,from,date
0,"삼성, 역성장 중남미 스마트폰 시장서도 1위 수성",조선비즈,2025.05.30 14:29
1,관세 충격 피한 북미 TV 시장…삼성·LG 일단 안도,이데일리,2025.05.30 14:17
2,[백종민의 딥테크]삼성이 포기한 서버로 富일군 대만,아시아경제,2025.05.30 14:14
3,삼성 스마트워치 1분기 출하량 전년比 74%↑… 저가·고가 제품 ‘투 ...,조선비즈,2025.05.30 14:07
4,"코스피, 외인 매도에 1%대 하락…코스닥도 재차 약세",이데일리,2025.05.30 13:56
5,'대어' 압구정2 놓고 삼성 vs 현대 재격돌…수주전 본격화(종합),연합뉴스,2025.05.30 13:17
6,‘국제학교 품은 학세권’ ‘평택 고덕 A48블록 금성백조 예미지’에.....,서울경제,2025.05.30 12:59
7,창업·스타트업 지원 공약…지식산업센터 호재될까,매일경제,2025.05.30 12:59
8,"""월급이 의느님 2배""...'삼성 고시'에 베트남 전국 수재들 다 모였...",파이낸셜뉴스,2025.05.30 12:08
9,"코스피 5000, 아주 불가능한 건 아니다? 대통령 누가돼도 우상향이라...",YTN,2025.05.30 11:52
